# MCP Weather Agent - Jupyter Notebook Version

This notebook demonstrates a LangChain agent that uses MCP (Model Context Protocol) tools instead of local tools.

## Setup
1. Make sure your MCP server is running
2. Set your OpenAI API key
3. Run the cells below

In [ ]:
# Install required packages
!pip install langchain langchain-openai python-dotenv fastmcp

In [ ]:
# Set your OpenAI API key
%env OPENAI_API_KEY='your-openai-api-key'

In [ ]:
import os
import asyncio
import json
from typing import Dict, Any
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain_core.prompts import PromptTemplate
from fastmcp import Client

# Load environment variables
load_dotenv()

# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# MCP Server URL 
MCP_SERVER_URL = "your-mcp-url"

print("✅ Imports and LLM initialized")

✅ Imports and LLM initialized


In [7]:
# Simple synchronous MCP client for Jupyter
def call_mcp_tool_sync(tool_name: str, arguments: Dict[str, Any]) -> str:
    """Synchronous wrapper for MCP tool calls"""
    async def _call_mcp_tool():
        try:
            async with Client(MCP_SERVER_URL) as client:
                print(f"🛠️ Calling MCP tool: {tool_name} with args: {arguments}")
                result = await client.call_tool(tool_name, arguments)
                
                if result.content and len(result.content) > 0:
                    result_text = result.content[0].text
                    try:
                        # Try to parse as JSON for structured formatting
                        result_data = json.loads(result_text)
                        return format_weather_result(tool_name, result_data, arguments)
                    except json.JSONDecodeError:
                        return result_text
                else:
                    return f"No result from {tool_name}"
        except Exception as e:
            return f"Error calling {tool_name}: {str(e)}"
    
    # Run the async function
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in Jupyter, create a new event loop in a thread
            import nest_asyncio
            nest_asyncio.apply()
            return loop.run_until_complete(_call_mcp_tool())
        else:
            return asyncio.run(_call_mcp_tool())
    except:
        # Fallback: create new event loop
        return asyncio.run(_call_mcp_tool())

def format_weather_result(tool_name: str, data: Dict[str, Any], arguments: Dict[str, Any]) -> str:
    """Format weather data for display"""
    if "error" in data:
        return f"Error: {data['error']}"
    
    location = arguments.get('location', 'Unknown')
    
    if tool_name == "get_current_weather":
        temp = data.get('temperature', 'N/A')
        desc = data.get('description', 'N/A').title()
        feels_like = data.get('feels_like', 'N/A')
        humidity = data.get('humidity', 'N/A')
        wind_speed = data.get('wind_speed', 'N/A')
        pressure = data.get('pressure', 'N/A')
        
        return f"""Current weather in {location}:
- Temperature: {temp}°C (feels like {feels_like}°C)
- Condition: {desc}
- Humidity: {humidity}%
- Wind Speed: {wind_speed} m/s
- Pressure: {pressure} hPa"""
        
    elif tool_name == "get_weather_forecast":
        forecast = data.get('forecast', [])
        days = len(forecast)
        result = f"{days}-day weather forecast for {location}:\n"
        
        for day in forecast:
            date = day.get('date', 'N/A')
            temp = day.get('temperature', 'N/A')
            desc = day.get('description', 'N/A').title()
            result += f"- {date}: {temp}°C, {desc}\n"
        
        return result
    
    # Generic formatting
    return json.dumps(data, indent=2)

print("✅ MCP client functions defined")

✅ MCP client functions defined


In [ ]:
# Install nest_asyncio for Jupyter compatibility
!pip install nest_asyncio

In [3]:
# Discover MCP tools and create LangChain tools
async def discover_mcp_tools():
    """Discover available tools from MCP server"""
    try:
        async with Client(MCP_SERVER_URL) as client:
            tools = await client.list_tools()
            print(f"🔗 Connected to MCP server! Found {len(tools)} tools:")
            for tool in tools:
                print(f"   • {tool.name}: {tool.description}")
            return tools
    except Exception as e:
        print(f"❌ Failed to connect to MCP server: {e}")
        return []

# Run discovery
import nest_asyncio
nest_asyncio.apply()

mcp_tools = await discover_mcp_tools()

🔗 Connected to MCP server! Found 2 tools:
   • get_current_weather: 
Get current weather for a specific location using OpenWeatherMap API 2.5.

Args:
    location: City name or location (e.g., "London", "New York", "Tokyo")
    
Returns:
    Dictionary with current weather information

   • get_weather_forecast: 
Get weather forecast for a specific location using OpenWeatherMap API 2.5.

Args:
    location: City name or location
    days: Number of days to forecast (max 5)
    
Returns:
    Dictionary with forecast information



In [4]:
# Create LangChain tools from MCP tools
def create_langchain_tool(mcp_tool):
    """Create a LangChain tool from an MCP tool"""
    def tool_function(tool_input: str) -> str:
        # Parse input - handle both JSON strings and plain strings
        try:
            if tool_input.startswith('{') and tool_input.endswith('}'):
                arguments = json.loads(tool_input)
            else:
                # Treat as location for weather tools
                arguments = {"location": tool_input.strip()}
        except json.JSONDecodeError:
            arguments = {"location": tool_input.strip()}
        
        return call_mcp_tool_sync(mcp_tool.name, arguments)
    
    return Tool(
        name=mcp_tool.name,
        description=mcp_tool.description or f"MCP tool: {mcp_tool.name}",
        func=tool_function
    )

# Create LangChain tools
langchain_tools = [create_langchain_tool(tool) for tool in mcp_tools]

print(f"✅ Created {len(langchain_tools)} LangChain tools from MCP server")

✅ Created 2 LangChain tools from MCP server


In [5]:
# Create the LangChain agent
template = '''You are a helpful weather assistant powered by MCP (Model Context Protocol). 
You provide weather-related information using remote weather intelligence tools.

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

agent = create_react_agent(
    tools=langchain_tools,
    llm=llm,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, 
    tools=langchain_tools, 
    verbose=True,
    handle_parsing_errors=True
)

print("✅ MCP Weather Agent ready!")

✅ MCP Weather Agent ready!


## Test the MCP Weather Agent

Now you can use the agent just like in your example! Use `agent_executor.invoke()` to ask weather questions:

In [8]:
# Test 1: Current weather
agent_executor.invoke({"input": "What is the weather in Oslo now?"})



> Entering new AgentExecutor chain...
I need to get the current weather information for Oslo. 
Action: get_current_weather
Action Input: "Oslo"🛠️ Calling MCP tool: get_current_weather with args: {'location': 'Oslo'}
Current weather in Oslo:
- Temperature: 8.64°C (feels like 8.64°C)
- Condition: Overcast Clouds
- Humidity: 92%
- Wind Speed: 0.95 m/s
- Pressure: 1032 hPaI now know the final answer. 
Final Answer: The current weather in Oslo is 8.64°C with overcast clouds, a humidity of 92%, and a wind speed of 0.95 m/s.

> Finished chain.


{'input': 'What is the weather in Oslo now?',
 'output': 'The current weather in Oslo is 8.64°C with overcast clouds, a humidity of 92%, and a wind speed of 0.95 m/s.'}

In [ ]:
# Test 2: Weather forecast
agent_executor.invoke({"input": "Get tomorrow's weather forecast for Oslo."})

## Key Differences from Local Tools

This notebook demonstrates:

1. **Dynamic Tool Discovery**: Tools are discovered from the MCP server automatically
2. **Remote Execution**: All weather logic runs on the MCP server
3. **Protocol Compliance**: Uses standard MCP protocol for communication
4. **Same Interface**: Works exactly like local tools from LangChain's perspective

The agent behaves identically to your local tools version, but all the weather intelligence comes from the remote MCP server!